Import 

In [1]:
import cv2 
import numpy as np
import random
import math

Some global values

In [2]:
img = cv2.imread("SAFlag.jpg")
#img = cv2.imread("baboon.jpg")
imgSize = [img.shape[0], img.shape[1]]
k =0
res = np.copy(img)
kpass = 0

Define trackbar

In [3]:
def k_val_trackbar(*arg):
    k = cv2.getTrackbarPos("value of k", "Kmeans")

Define function to active algorithm

In [4]:
def active():
    k = cv2.getTrackbarPos("value of k", "Kmeans")
    storeimg(imgSize)
    Kmeans(img, imgSize, k)

Two functions to calculate how close two pixels are

In [5]:
def pixelDiff(p1, p2):        
    val = squrediffernce(p1[0], p2[0])+squrediffernce(p1[1], p2[1])+squrediffernce(p1[2], p2[2])  
    return val
        
def squrediffernce(x, y):                      
    return math.sqrt(math.pow(x-y, 2))


Get random k centers

In [6]:
def RandomCenter(img, imgSize, k):
    centers = np.zeros((k,img.shape[2]))                   
    for i in range(k):                           #random pick k points in the img
        x = random.randint(0, imgSize[0]-1)
        y = random.randint(0,imgSize[1]-1)
        centers[i,: ] = np.copy(img[x][y])     # store the BGR value of each center points
    return centers           


An function that can store the original image so that trackbar can be used continuously

In [7]:
def storeimg(imgSize):  
    for x in range(imgSize[0]):
        for y in range(imgSize[1]):
            img[x][y] = res[x][y]

K-means algorithm

In [8]:
def Kmeans(img, imgSize, k):   

    VRS = np.zeros((imgSize[0],imgSize[1], 2))        # [x,y,0], stores cluster type(different centers)
                                                    #  [x,y,1], stores the difference between sample pixel and centers 
    centers = RandomCenter(img, imgSize,k)          # get center points
    controller = True                                 
    threshold = 5                             # the value to check how close between two points
    while(controller==True):

        controller = False
        for x in range(imgSize[0]):
            for y in range(imgSize[1]):
                standarddiff = 1000**3    # give a large standard difference to test how close two pixels are, I will update this value! 
                minIdx = 0
                for i in range(k):
                    tempDiff = pixelDiff(centers[i], img[x][y])  #get the pixel difference between two points
                    difference = squrediffernce(tempDiff, standarddiff)  # get the difference two difference
                    if tempDiff < standarddiff and squrediffernce(tempDiff, standarddiff) > threshold :
                        standarddiff = tempDiff          #update the standard difference
                        minIdx = i                       # index different clusters with different centers
                VRS[x,y,:] = [minIdx,tempDiff] #store the cluster type and difference
        
        means = []        # stack of mean
        diff = []         #stack of difference
        test = False            # test different situation
        
        for i in range(k):
            cluster = (VRS[:,:,0] == i)      # get cluster type
            clusterPVal = img[cluster]       # cluster pixel value
            means.append(np.mean(clusterPVal, axis=0))          
        for j in range(len(centers)):
            tempDiff = pixelDiff(centers[j], means[j]) # get difference
            diff.append(tempDiff)            #store difference
            if  tempDiff >= threshold:             
                test = True
        if test == False:
            controller = False
        else:
            controller = True
            centers = means

        for x in range(imgSize[0]):        #change the color in clusters
            for y in range(imgSize[1]):
                idx = int(VRS[x,y,0])
                img[x,y] = centers[idx]

Main function(Please press "enter" after you have chosen a k value)

In [9]:
cv2.namedWindow("Kmeans", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Kmeans",imgSize[1], imgSize[0])
cv2.createTrackbar("value of k", "Kmeans", 0,30, k_val_trackbar)
while True:
    cv2.imshow("Kmeans", img)
    key = cv2.waitKey(1)
    if key == 27:   # "esc" to exit
        break
    if key== 13:   #"enter" to active the alogrithm
        active()
cv2.destroyAllWindows()   